In [61]:
import os, json
import pandas as pd, numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from collections import Counter




# this finds our json files
path_to_json = 'data_folder'

stop_list = list(STOPWORDS) + ["sil", "uh"]

VOCAB = set()
score_dict = {}
tot = {}
voc_dict = {}
vocab_list = []


new_dict = {}
new_dict["f_1"] = []
new_dict["f_2"] = []
new_dict["f_3"] = []
new_dict["f_4"] = []
new_dict["scores"] = []
new_dict["binary_label"] = []
new_dict["multiclass_label"] = []

data = []

setter = set()
inter = set()
# we need both the json and an index number so use enumerate()
for subdir, dirs, files in os.walk(path_to_json):
    for file in files:
        if file.endswith(".json"):
            path = os.path.join(subdir, file)
    
            with open(path, 'r') as f:
                json_text = json.load(f)
            id_ = json_text["id"]
            feature_dict = {}

            score = json_text["score"]

            if score < 5:
                print("OUTLIER")
                continue

            new_dict["scores"].append(score)


            text = ""
            doc_vocab = set()
            counter = 0
            for tok in json_text["tokens"]:
                Text = tok["text"].lower()
                if Text not in stop_list:
                    text += " " + Text
                    counter += 1
                    doc_vocab.add(Text)
                    VOCAB.add(Text)
                    vocab_list.append(Text)

            data.append(text)


#            for stop_word in stop_list:
#                try:
#                    doc_vocab.remove(stop_word)
#                    VOCAB.remove(stop_word)
#               except: continue
    
            # new words pr min
            f_1 = len(doc_vocab)/json_text["elapsed_time"]
            if (f_1 < 0.15):
                print('ERRORRRRRR')
                print(subdir)


            feature_dict["new_words_pr_min"] = f_1


            # repeated words pr min
            f_2 = (counter - len(doc_vocab))/json_text["elapsed_time"]
            feature_dict["repeated_words_pr_min"] = f_2
            
            new_dict["f_1"].append(f_1)
            new_dict["f_2"].append(f_2)
    
            
            feature_dict["time"] = json_text["elapsed_time"]
            tot[id_] = feature_dict
    
            voc_dict[id_] = doc_vocab
    
            union = setter.union(doc_vocab-inter)
            intersect = setter.intersection(doc_vocab)
            setter = union - intersect
            inter = intersect
            

            # Labelling process

            # Goal: set thresholds to get uniform distribution
            if score > 91:
                new_dict["binary_label"].append(1)
                if score > 95.5:
                    new_dict["multiclass_label"].append(3)
                else:
                    new_dict["multiclass_label"].append(2)
            else:
                new_dict["binary_label"].append(0)
                if (score < 91) & (score > 79):
                    new_dict["multiclass_label"].append(1)
                else:
                    new_dict["multiclass_label"].append(0)


# f_3
for id_ in tot.keys():
    voc = voc_dict[id_]
    time = tot[id_]["time"]
    f_3 = len(voc.intersection(setter))/(time)
    new_dict["f_3"].append(f_3)

# f_4
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data)
#print(vectorizer)
#print(X)
transformer = TfidfTransformer()
data = transformer.fit_transform(X).toarray()
#print(data)

summed_data = np.sum(data, axis=1)
new_dict["f_4"] = summed_data




print("create dataframe from dictionary")
df = pd.DataFrame.from_dict(new_dict)

print("Normalizing data")
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        if "f_" in feature_name:
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

df = normalize(df)


#print(df)
#print(df)

# print("Preprocessing done")
# print("Preparing classification task")


# print("Data Exploration")

# # Data exploration

# import matplotlib.pyplot as plt
# print("Histograms")

# df["scores"].hist(bins = 100)
# plt.title("Gweek Score Histogram, 492 datapoints")
# plt.show()

# df["binary_label"].hist(bins = 2)
# plt.title("Binary Label Histogram, 492 datapoints, (<95/>95)")
# plt.show()

# df["multiclass_label"].hist(bins = 4)
# plt.title("Multiclass Label Histogram, 4 classes, 492 datapoints")
# plt.show()
# #############
# print("Correlation plots")
# plt.plot(df["f_1"], df["scores"], 'bo')
# plt.xlabel('Distinct words pr sek')
# plt.ylabel('Scores')
# plt.title('Correlation plot, 492 datapoints')
# plt.show()

# plt.plot(df["f_1"], df["f_3"], 'bo')
# plt.xlabel('Distinct words pr sek')
# plt.ylabel('Special words')
# plt.title('Correlation plot, 492 datapoints')
# plt.show()

# plt.plot(df["f_1"], df["f_4"], 'bo')
# plt.xlabel('Distinct words pr sek')
# plt.ylabel('TF IDF')
# plt.title('Correlation plot, 492 datapoints')
# plt.show()

# plt.plot(df["f_4"], df["scores"], 'bo')
# plt.xlabel('TF IDF')
# plt.ylabel('Gweek Score')
# plt.title('Correlation plot, 492 datapoints')
# plt.show()

# ###########Word Cloud###################

# dicter = Counter(vocab_list)

# wordcloud = WordCloud(
#     background_color='white',
#     stopwords=stop_list,
#     max_words=200,
#     max_font_size=80,
#     random_state=42
# ).generate_from_frequencies(dicter)

# fig = plt.figure(1)
# plt.imshow(wordcloud)
# plt.title("Word Cloud, 492 datapoints")
# plt.axis('off')
# #plt.show()


# # Training

print("--------------------------------------")
print("Preparing binary classification task")

y = df.iloc[:,-2]
X = df.iloc[:,:-3]


# # Training -------------------------------------
# # splitting
from sklearn.model_selection import train_test_split
a_train, a_test, b_train, b_test = train_test_split(X, y, test_size=0.33, random_state=42)

#print(b_train)
############Random Forest classifier#####################

# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier()
# rf.fit(a_train, b_train);
# predictions = rf.predict(a_test)
# #Import scikit-learn metrics module for accuracy calculation
# from sklearn import metrics
# # Model Accuracy, how often is the classifier correct?
# print("Accuracy for RandomForestClassifier:",metrics.accuracy_score(b_test, predictions))

# ######################## Naive Bayes classifier###############

# #Import Gaussian Naive Bayes model
# from sklearn.naive_bayes import GaussianNB

# #Create a Gaussian Classifier
# gnb = GaussianNB()

# #Train the model using the training sets
# gnb.fit(a_train, b_train)

# #Predict the response for test dataset
# prediction = gnb.predict(a_test)

# from sklearn import metrics
# # Model Accuracy, how often is the classifier correct?
# print("Accuracy for Naive Bayes Classifier:",metrics.accuracy_score(b_test, prediction))

########################## Gradient Classifier###################

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(a_train, b_train)
print("Accuracy score (training): {0:.3f}".format(gb.score(a_train,b_train)))
print("Accuracy score (test): {0:.3f}".format(gb.score(a_test, b_test)))

# from sklearn.linear_model import LogisticRegression


# LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(a_train, b_train)
# predicted = LR.predict(a_test)
# print("F1-score BINARY")
# print(f1_score(predicted, np.array(b_test), average='weighted'))
# #print(round(LR.score(predicted, np.array(b_test))))

# print("--------------------------------------")
# print("Preparing multiclass classification task")

# y = df.iloc[:,-1]
# #X = df.iloc[:,:-3]


# # Training -------------------------------------
# # splitting
# a_train, a_test, b_train, b_test = train_test_split(X, y, test_size=0.33, random_state=42)


# LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(a_train, b_train)
# predicted = LR.predict(a_test)
# print("F1-score MULTICLASS")
# print(f1_score(predicted, np.array(b_test), average='weighted'))
# #print(round(LR.score(predicted, np.array(b_test))))




# print("--------------------------------------")
# print("Preparing regression task")
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error
# from math import sqrt

# y = df.iloc[:,-3]

# a_train, a_test, b_train, b_test = train_test_split(X, y, test_size=0.33, random_state=42)
# LinR = LinearRegression().fit(a_train, b_train)
# predicted = LinR.predict(a_test)
# rmse = sqrt(mean_squared_error(predicted, b_test))
# print("Root Mean square error")
# print(rmse)





OUTLIER
create dataframe from dictionary
Normalizing data
--------------------------------------
Preparing binary classification task
Accuracy score (training): 1.000
Accuracy score (test): 0.583
